Generate file with subcolumns

In [1]:
import pandas as pd


def generate_filtered_bulk(df, bulk_df, idx_name):
    data_field = bulk_df['data-field'][0].split('_')[0]
    filename = 'ukb_filtered_' + data_field + '.bulk' 

    # get all ieds from bulk file
    datafield_ieds = list(set(bulk_df.iloc[:, 0]))
    # get subset of filtered dataset which overlaps with bulk ieds
    # print(datafield_ieds)
    df_fc_filtered = df[df['eid'].isin(datafield_ieds)]
    # remove invalid samples
    df_fc_filtered = df_fc_filtered[df_fc_filtered['gender'] != '-1']
    # generate bulk dataframe
    filtered_eids = list(df_fc_filtered['eid'])
    bulk_df = bulk_df[bulk_df['eid'].isin(filtered_eids)]

    # bulk_df.to_csv(filename, sep=' ', index=False, header=False)
    # filename = 'ukb_filtered_' + data_field + '_harir_mh_upto69.csv'
    filename = "rds_over_8_dup.csv"
    labels = df_fc_filtered[['eid', 'gender','age', idx_name,'rds4 >8']]
    # remove columns with duplicate eids
    # labels = labels.drop_duplicates(subset='eid')
    # labels.to_csv(filename, sep = ' ', index=False)
    return bulk_df, labels
    


df2 = pd.read_csv('ukb_filtered_hariri_mh_upto69_v2.csv', sep=',')
df = pd.read_csv('ukb_filtered_hariri_mh_upto69.csv', sep=' ')

df_fc = pd.read_csv('ukb675871_25753.bulk', sep=' ', names = ['eid', 'data-field'])

bulk_df, labels = generate_filtered_bulk(df, df_fc, 'Lifetime dep Smith')
bulk_df2, labels2 = generate_filtered_bulk(df2, df_fc, idx_name= 'Lifetime depression Smith')

print(bulk_df.head())
print(labels.head())

eids = labels['eid'].unique()
eids2 = labels2['eid'].unique()
print(len(eids))
print(len(eids2))   

# get overlap between eids and eids2
overlap = list(set(eids).intersection(eids2))
print(len(overlap))

       eid data-field
0  1000032  25753_2_0
2  1000382  25753_2_0
3  1000382  25753_3_0
5  1000457  25753_2_0
9  1000756  25753_2_0
       eid  gender   age Lifetime dep Smith  rds4 >8
0  1000032       1  67.0                 HC        0
1  1000382       1  51.0                 HC        0
2  1000382       1  53.0                 HC        0
3  1000457       0  66.0                 HC        0
4  1000756       0  51.0                 HC        0
13093
16709
13093


In [2]:
print(df2['rds4 >8'].value_counts())
print(df['rds4 >8'].value_counts())
# get eids where df2['rds4 >8'] == -1
df2eids = df2[df2['rds4 >8'] == 0]['eid']
print(len(eids))


# get eids where df2['rds4 >8'] == 0
dfeids = df[df['rds4 >8'] == 1]['eid']

# get overlap between eids 
overlap = list(set(dfeids).intersection(df2eids)) 
print(len(overlap))


 0    17267
-1      830
 1      798
Name: rds4 >8, dtype: int64
0    14207
1      596
Name: rds4 >8, dtype: int64
13093
80


Generate balanced file with eid + labels matched on gender and age

In [10]:
df = pd.read_csv('rds_over_8.csv', sep=' ')

# eids = list(df['eid'])
print(len(eids))
print(len(set(eids)))

df.head()
 

1150
1087


,eid,gender,age,Lifetime depression Smith,rds4 >8
0,1000032,1,67.0,0.HC,0
1,1000315,0,59.0,-1,0
2,1000382,1,51.0,0.HC,0
3,1000457,0,66.0,0.HC,0
4,1000537,1,61.0,-1,0


In [11]:
# rds_sub_8_eids =  [sample[0] for sample in eids_smith_list]

# # get columns of df with eid values from rds_sub_8_eids
# df_rds_sub_8 = df[df['eid'].isin(rds_sub_8_eids)]

# # check rds4 > 8 value counts
# df_rds_sub_8['rds4 >8'].value_counts()

# # get eids where rd4 > 8
# rds_over_8_eids = list(df_rds_sub_8[df_rds_sub_8['rds4 >8'] == 1]['eid'])
# rds_over_8_eids

In [12]:
# create subset with variable depr where rds4 > 8 and lifetime depression

# print(df['Lifetime depression Smith'].value_counts())
depr = df[(df['rds4 >8'] == 0) & (df['Lifetime depression Smith'] == '3.Severe rMDD')]
# print(depr.head(1))
hc = df[(df['rds4 >8'] == 0) & (df['Lifetime depression Smith'] == '0.HC')]
# hc = df[df['rds4 >8'] == 0 ]
# hc = df[df['Lifetime dep Smith'] == 'HC']
# print(depr['Lifetime dep Smith'].value_counts())

print(depr['Lifetime depression Smith'].value_counts())
print(hc['Lifetime depression Smith'].value_counts())
depr_eids = list(depr['eid'])
hc_eids = list(hc['eid'])

print(len(eids))
# check if depr eids are all in eids

print(len(depr))
rem = []
for eid in depr_eids:
    if eid not in eids:
        # remove eid from depr dataframe
        rem.append(eid)
        depr = depr[depr['eid'] != eid]
print(len(depr))

print(len(hc))
for eid in hc_eids:
    if eid not in eids:
        hc = hc[hc['eid'] != eid]
        rem.append(eid)
print(len(hc))

# check if 1001353 is in rem
print(1001353 in rem)

3.Severe rMDD    516
Name: Lifetime depression Smith, dtype: int64
0.HC    9119
Name: Lifetime depression Smith, dtype: int64
1150
516
515
9119
544
True


In [13]:
import numpy as np
"""
matches = []

# Loop over each row in the subset to find the nearest HC match
i = 0
for _, row in depr.iterrows():
    # Filter HC group for the same gender
    gender_matches = hc[hc['gender'] == row['gender']]
    
    # Calculate absolute age difference
    gender_matches['age_diff'] = np.abs(gender_matches['age'] - row['age'])
    
    # Find the index of the smallest age difference
    best_match_index = gender_matches['age_diff'].idxmin()
    
    # Append the best match to the matches list
    
    matches.append(hc.loc[best_match_index])
    i+=1

# Convert the list of matches to a DataFrame
matched_df = pd.DataFrame(matches)

print(matched_df)

matched_eids = list(matched_df['eid'])
print(len(matched_eids))
print(len(set(matched_eids)))
print(i)
"""

def match_df(depr, hc):
    matches = []

    # Loop over each row in the subset to find the nearest HC match
    i = 0
    for _, row in depr.iterrows():
        # find gender of instance

        gender_matches = hc[hc['gender'] == row['gender']]

        gender_matches['age_diff'] = np.abs(gender_matches['age'] - row['age'])
    
        # Find the index of the smallest age difference
        best_match_index = gender_matches['age_diff'].idxmin()
        
        # Append the best match to the matches list
        matches.append(hc.loc[best_match_index])

        # remove matched HC from the pool
        hc = hc.drop(best_match_index)

    return pd.DataFrame(matches)


matched_df = match_df(depr, hc)


matched_eids = list(matched_df['eid'])
print(len(matched_eids))
print(len(set(matched_eids)))


/home/ljdevries/tmp/ipykernel_28458/2911561518.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_matches['age_diff'] = np.abs(gender_matches['age'] - row['age'])
/home/ljdevries/tmp/ipykernel_28458/2911561518.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_matches['age_diff'] = np.abs(gender_matches['age'] - row['age'])
/home/ljdevries/tmp/ipykernel_28458/2911561518.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

515
515


/home/ljdevries/tmp/ipykernel_28458/2911561518.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_matches['age_diff'] = np.abs(gender_matches['age'] - row['age'])
/home/ljdevries/tmp/ipykernel_28458/2911561518.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_matches['age_diff'] = np.abs(gender_matches['age'] - row['age'])
/home/ljdevries/tmp/ipykernel_28458/2911561518.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [14]:

combined_df = pd.concat([depr, matched_df], ignore_index=True)

# Shuffle the combined DataFrame
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

print(combined_df['Lifetime depression Smith'].value_counts())
print(combined_df['rds4 >8'].value_counts())

0.HC             515
3.Severe rMDD    515
Name: Lifetime depression Smith, dtype: int64
0    1030
Name: rds4 >8, dtype: int64


In [15]:
eid_rds_list = list(combined_df[['eid', 'rds4 >8']].itertuples(index=False, name=None))

eids = [eid for eid, rds in eid_rds_list]
print(len(eids))
print(len(set(eids)))

print(eid_rds_list)
print(len(eid_rds_list))

1030
1030
[(1145226, 0), (1169361, 0), (3664111, 0), (1183595, 0), (6009361, 0), (3318260, 0), (4488609, 0), (1055345, 0), (1222606, 0), (1398221, 0), (1541360, 0), (5889730, 0), (4950699, 0), (1076385, 0), (4456812, 0), (1107750, 0), (1391263, 0), (4982689, 0), (1205611, 0), (1370512, 0), (1234070, 0), (1560963, 0), (1364219, 0), (1144121, 0), (1169111, 0), (1066369, 0), (1509499, 0), (1191246, 0), (1123302, 0), (3066463, 0), (1500535, 0), (5005010, 0), (1370306, 0), (5057978, 0), (3254941, 0), (5004537, 0), (1922013, 0), (1127046, 0), (1078492, 0), (1456714, 0), (3992533, 0), (1209967, 0), (4728728, 0), (1160837, 0), (5774146, 0), (1064723, 0), (1086996, 0), (4862523, 0), (3614262, 0), (3087837, 0), (5525112, 0), (3925965, 0), (3586847, 0), (3541030, 0), (5180805, 0), (3465361, 0), (1007443, 0), (5193682, 0), (1177052, 0), (1181337, 0), (2540698, 0), (1114056, 0), (5398523, 0), (1037759, 0), (2743166, 0), (2029868, 0), (4755917, 0), (1138360, 0), (5902994, 0), (2076852, 0), (3131929,

In [16]:


# replace everything HC with 0 and everything else with 1

print(combined_df['Lifetime depression Smith'].value_counts())
combined_df['Lifetime depression Smith'] = combined_df['Lifetime depression Smith'].replace('0.HC', 0)
# combined_df['Lifetime depression Smith'] = combined_df['Lifetime depression Smith'].replace('1.Single ep.', 1)
# combined_df['Lifetime depression Smith'] = combined_df['Lifetime depression Smith'].replace('2.Moderate rMDD', 1)
combined_df['Lifetime depression Smith'] = combined_df['Lifetime depression Smith'].replace('3.Severe rMDD', 1)


# print(eids_smith_list)


print(combined_df['Lifetime depression Smith'].value_counts())
eids_smith_list = list(combined_df[['eid', 'Lifetime depression Smith']].itertuples(index=False, name=None))

print(eids_smith_list)
len(eids_smith_list)



0.HC             515
3.Severe rMDD    515
Name: Lifetime depression Smith, dtype: int64
0    515
1    515
Name: Lifetime depression Smith, dtype: int64
[(1145226, 0), (1169361, 0), (3664111, 1), (1183595, 0), (6009361, 1), (3318260, 1), (4488609, 1), (1055345, 0), (1222606, 0), (1398221, 0), (1541360, 1), (5889730, 1), (4950699, 1), (1076385, 0), (4456812, 1), (1107750, 0), (1391263, 0), (4982689, 1), (1205611, 0), (1370512, 0), (1234070, 0), (1560963, 0), (1364219, 0), (1144121, 0), (1169111, 0), (1066369, 0), (1509499, 1), (1191246, 0), (1123302, 0), (3066463, 1), (1500535, 1), (5005010, 1), (1370306, 0), (5057978, 1), (3254941, 1), (5004537, 1), (1922013, 1), (1127046, 0), (1078492, 0), (1456714, 1), (3992533, 1), (1209967, 0), (4728728, 1), (1160837, 0), (5774146, 1), (1064723, 0), (1086996, 0), (4862523, 1), (3614262, 1), (3087837, 1), (5525112, 1), (3925965, 1), (3586847, 1), (3541030, 1), (5180805, 1), (3465361, 1), (1007443, 0), (5193682, 1), (1177052, 0), (1181337, 0), (254069

1030

In [9]:
# Write the list of matched EIDs to a file

with open('severe_rds_v2.csv', 'w') as f:
    for item in eid_rds_list:
        f.write("%s %s\n" % item)



In [17]:
with open('rds_under_8_severe_v2.csv', 'w') as f:
    for item in eids_smith_list:
        f.write("%s %s\n" % item)